In [1]:
import torch, torchvision
from torch import nn
from torch import optim
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO
from torchvision.datasets import ImageFolder


import copy
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from googlenet_pytorch import GoogLeNet

In [ ]:
numb_batch = 2

In [ ]:
T = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])
# train_data = torchvision.datasets.MNIST('mnist_data', train=True, download=True, transform=T)
# val_data = torchvision.datasets.MNIST('mnist_data', train=False, download=True, transform=T)

train_data = ImageFolder(r'D:\nanodataset\training',transform=T)
val_data = ImageFolder(r'D:\nanodataset\validation',transform=T)
test_data = ImageFolder(r'D:\nanodataset\testing',transform=T)
# train_dl = torch.utils.data.DataLoader(train_data, batch_size = numb_batch)
# val_dl = torch.utils.data.DataLoader(val_data, batch_size = numb_batch)
train_dl=DataLoader(train_data, batch_size=numb_batch)
val_dl=DataLoader(val_data, batch_size=numb_batch)
test_dl=DataLoader(test_data, batch_size=numb_batch)

In [ ]:

model = GoogLeNet.from_pretrained("googlenet")
model.eval()

In [ ]:
def validate(model, data):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(data):
        images = images.cuda()
        x = model(images)
        value, pred = torch.max(x,1)
        pred = pred.data.cpu()
        total += x.size(0)
        correct += torch.sum(pred == labels)
    return correct*100./total

In [ ]:
def train(numb_epoch=3, lr=1e-3, device="cpu"):
    accuracies = []
    cnn = model.to(device)
    cec = nn.CrossEntropyLoss()
    optimizer = optim.Adam(cnn.parameters(), lr=lr)
    max_accuracy = 0
    for epoch in range(numb_epoch):
        for i, (images, labels) in enumerate(train_dl):
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            pred = cnn(images)
            loss = cec(pred, labels)
            loss.backward()
            optimizer.step()
        accuracy = float(validate(cnn, val_dl))
        accuracies.append(accuracy)
        if accuracy > max_accuracy:
            best_model = copy.deepcopy(cnn)
            max_accuracy = accuracy
            print("Saving Best Model with Accuracy: ", accuracy)
        print('Epoch:', epoch+1, "Accuracy :", accuracy, '%')
    plt.plot(accuracies)
    return best_model

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("No Cuda Available")
device

In [ ]:
lenet = train(10, device=device)

In [ ]:
#Confusion matrix
def predict_dl(model, data):
    y_pred = []
    y_true = []
    for i, (images, labels) in enumerate(data):
        images = images.cuda()
        x = model(images)
        value, pred = torch.max(x, 1)
        pred = pred.data.cpu()
        y_pred.extend(list(pred.numpy()))
        ##.extend is used to iterate
        ##.numpy to convert tensor in numpyarray
        y_true.extend(list(labels.numpy()))
    return np.array(y_pred), np.array(y_true)

In [ ]:
y_pred, y_true = predict_dl(lenet, test_dl)

In [ ]:
pd.DataFrame(confusion_matrix(y_true, y_pred, labels=np.arange(0,4)))
